In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
df = pd.read_csv(io.BytesIO(uploaded['qoute_dataset.csv']))

Saving qoute_dataset.csv to qoute_dataset.csv


In [ ]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [ ]:
df.shape

(3038, 2)

In [ ]:
df['quote'][2]

'“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'

In [ ]:
df.describe()

,quote,Author
count,3038,3038
unique,3037,1005
top,A woman's heart should be so hidden in God tha...,"Cassandra Clare,"
freq,2,92


In [ ]:
#OBJECTIVE-> find the next word of the quote that one would be writing.
# so jo krna h quotes ke saath krna h author ka koi vaasta nhi h

In [ ]:
quotes = df['quote']
quotes.head()

,quote
0,“The world as we have created it is a process ...
1,"“It is our choices, Harry, that show what we t..."
2,“There are only two ways to live your life. On...
3,"“The person, be it gentleman or lady, who has ..."
4,"“Imperfection is beauty, madness is genius and..."


In [ ]:
#Preprocessing

#lowercase krlete h pehle sabkuch
import string
quotes = quotes.apply(lambda x: x.lower())
quotes.head()

,quote
0,“the world as we have created it is a process ...
1,"“it is our choices, harry, that show what we t..."
2,“there are only two ways to live your life. on...
3,"“the person, be it gentleman or lady, who has ..."
4,"“imperfection is beauty, madness is genius and..."


In [ ]:
#ab comma vgera hatedete hai
quotes = quotes.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
smart_quotes = '“”‘’"'
quotes = quotes.apply(lambda x: str(x).translate(str.maketrans('', '', smart_quotes)).strip().lower())

quotes.head()

,quote
0,the world as we have created it is a process o...
1,it is our choices harry that show what we trul...
2,there are only two ways to live your life one ...
3,the person be it gentleman or lady who has not...
4,imperfection is beauty madness is genius and i...


In [ ]:
# ab tokenize krte h words ko
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(quotes) #isne indexing krdi saare words ki as a dict

In [ ]:
#now just to check ki jo hua kya hua we using .word_index; it's not part of process.
word_index = tokenizer.word_index
print(len(word_index))
list(word_index.items())[:10]

8769


[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10)]

In [ ]:
# ab in saare words o=ko hi sentence me bithaado to make taht D1,D2(ref to notes)
sequence = tokenizer.texts_to_sequences(quotes)

In [ ]:
quotes[0]

'the world as we have created it is a process of our thinking it cannot be changed without changing our thinking'

In [ ]:
sequence[0]

[1,
 62,
 29,
 19,
 16,
 939,
 10,
 7,
 5,
 1143,
 8,
 70,
 285,
 10,
 146,
 12,
 805,
 105,
 743,
 70,
 285]

In [ ]:
print(tokenizer.word_index.get('world'))

62


In [ ]:
print("ID for 'the':", tokenizer.word_index.get('the'))

ID for 'the': 1


In [ ]:
# ab merko kranah predict to isme x,y kya hoga ek hi to h apne pe to
# logic idhar ye h ki if i write the to wo world likhde ya if i wrote teh world to vo as likhde and so on

X=[]
y=[]

for seq in sequence:
  for i in range(1,len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

In [ ]:
len(X)

85269

In [ ]:
len(y)

85269

In [ ]:
#ab hume input size same chahiye , just print(X) and u wud see diff lne ke sentences but we want sbki size same ho;
# to ham padding add krdenge start me eg-> D1-> 126 only and max len saare sentences dekhke let's say is 3 so D1->[0,0,126].

# Neural networks perform calculations using Matrix Multiplication. For the computer to multiply the input data by the "weights" of the model, the input needs to have a fixed shape.


# If one row has 5 numbers and the next has 20, the computer doesn't know how to align them for the math equations.

# By using Padding, you turn a "jagged" list of varying lengths into a perfect rectangle.

In [ ]:
max_len = max(len(x) for x in X)
max_len

745

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')

In [ ]:
X_padded

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,   62],
       [   0,    0,    0, ...,    1,   62,   29],
       ...,
       [   0,    0,    0, ...,    9,   19, 1112],
       [   0,    0,    0, ...,   19, 1112,    3],
       [   0,    0,    0, ..., 1112,    3,  172]], dtype=int32)

In [ ]:
# y ko bhi array me krlo ab
y=np.array(y)

In [ ]:
#ab one-hot encoding krdete h y ki bcs obv
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size) # we're having 10k and not 8769 here bcs sbse pehle kabhibhi exact number nhi maye +2 rkho and secondly it's fine to ahev 10k bcs baadme agar or bhd bhi gyi vocab to arch change nhi krna pdega apnako

In [ ]:
# AB EMBEDDING KRTE H
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN

In [ ]:
# Abhi tak aapke words sirf numbers hain (e.g., "the" = 1, "world" = 62). Computer ke liye 1 aur 62 ke beech mein koi rishta (relationship) nahi hai. Embedding Dimension woh tarika hai jisse hum computer ko words ka "matlab" aur unka "connection" samjhate hain.

embed_dim = 64
rnn_NoOfNeuorns = 128

In [ ]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len))
rnn_model.add(SimpleRNN(units = rnn_NoOfNeuorns))
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#ab ek lstm ka model krlete h
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len))
lstm_model.add(LSTM(units = rnn_NoOfNeuorns))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ab krte h fit data in model
history_rnn = rnn_model.fit(
    X_padded,
    y_one_hot,
    batch_size=32,
    epochs=100,
    validation_split=0.1
)

Epoch 1/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 148s 59ms/step - accuracy: 0.0348 - loss: 7.2347 - val_accuracy: 0.0338 - val_loss: 7.1842
Epoch 2/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 140s 58ms/step - accuracy: 0.0463 - loss: 6.7137 - val_accuracy: 0.0620 - val_loss: 6.7110
Epoch 3/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 139s 58ms/step - accuracy: 0.0804 - loss: 6.2291 - val_accuracy: 0.0871 - val_loss: 6.6051
Epoch 4/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 139s 58ms/step - accuracy: 0.1000 - loss: 5.9585 - val_accuracy: 0.1010 - val_loss: 6.5791
Epoch 5/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 139s 58ms/step - accuracy: 0.1153 - loss: 5.7633 - val_accuracy: 0.0944 - val_loss: 6.4944
Epoch 6/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 142s 58ms/step - accuracy: 0.1143 - loss: 5.8935 - val_accuracy: 0.1009 - val_loss: 6.6333
Epoch 7/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 139s 58ms/step - accuracy: 0.1290 - loss: 5.6047 - val_accuracy: 0.1010 - val_loss: 6.6194
Epoch 8/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 140s 58ms/step - ac

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
rnn_model = pd.read_csv(io.BytesIO(uploaded['rnn_model.h5']))

Saving rnn_model.h5 to rnn_model.h5


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [ ]:
rnn_model.save('rnn_model.h5')

In [ ]:
#lstm model
history_lstm = lstm_model.fit(X_padded, y_one_hot, batch_size=32, epochs=100, validation_split=0.1)

Epoch 1/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 2712s 1s/step - accuracy: 0.0386 - loss: 6.8790 - val_accuracy: 0.0559 - val_loss: 6.5620
Epoch 2/100
2399/2399 ━━━━━━━━━━━━━━━━━━━━ 2657s 1s/step - accuracy: 0.0657 - loss: 6.1611 - val_accuracy: 0.0860 - val_loss: 6.4242
Epoch 3/100
   6/2399 ━━━━━━━━━━━━━━━━━━━━ 38:25 963ms/step - accuracy: 0.1016 - loss: 5.9679

In [ ]:
from tensorflow.keras.models import load_model

# This loads the actual architecture and weights from the file
rnn_trained_model = load_model('rnn_model.h5')

# Now run this to check if it's back
rnn_trained_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 745, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10000)          │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,954,706 (7.46 MB)

 Trainable params: 1,954,704 (7.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
# ab dekho apan ne o/p number me niklaa hai but word dena haina apn ko to o/p me
index_to_word = {}
for word,index in word_index.items():
  index_to_word[index] = word
# hence now we have index as key and value as word

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def predictor(model,tokenizer,text,max_len):
  text = text.lower()

  seq = tokenizer.texts_to_sequences([text])[0]
  seq = pad_sequences([seq], maxlen=max_len, padding='pre')

  pred = model.predict(seq, verbose=0)
  pred_word_index = np.argmax(pred)
  return index_to_word[pred_word_index]

In [ ]:
seed_text = "life is"
next_word = predictor(rnn_trained_model,tokenizer,seed_text,max_len)
print(seed_text,next_word)

life is a


In [ ]:
seed_text = "life is a"
next_word = predictor(rnn_trained_model,tokenizer,seed_text,max_len)
print(seed_text,next_word)

life is a warm
